In [409]:
import pandas as pd
from nltk.corpus import stopwords 
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_predict
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score

In [410]:
RANDOM_SEED = 13

In [411]:
df = pd.read_excel("sentencas.xlsx")
df.head()

,Intenção,Sentença
0,Consultar saldo da conta,Qual o meu saldo
1,Interagir com a luz ou o ar-condicionado,Ligue a luz
2,Consultar saldo da conta,Me diga meu saldo
3,Consultar saldo da conta,Qual o meu saldo?
4,Interagir com a luz ou o ar-condicionado,está muito claro


## Limpando o dataset

In [412]:
df["Sentença"] = df["Sentença"].str.lower()
df["Sentença"] = df["Sentença"].str.replace(r"\s\-\s|\-\-+", " ", regex=True)
df["Sentença"] = df["Sentença"].str.replace(r"[^\w\s\-]", " ", regex=True)
df["Sentença"] = df["Sentença"].str.replace("foxbot ", "", regex=False)
df.head()

,Intenção,Sentença
0,Consultar saldo da conta,qual o meu saldo
1,Interagir com a luz ou o ar-condicionado,ligue a luz
2,Consultar saldo da conta,me diga meu saldo
3,Consultar saldo da conta,qual o meu saldo
4,Interagir com a luz ou o ar-condicionado,está muito claro


## Adequação dos dados para o MultinomialNB

In [413]:
# Adequa os dados para entrada no modelo MultinomialNB e já separa em X e y.
v = CountVectorizer()
def adaptToModel(df, objective=None, vectorizer=v):
    txts = df["Sentença"].tolist()
    if objective == "train":
        counts = vectorizer.fit_transform(txts)
        return counts, df["Intenção"]
    elif objective == "test":
        counts = vectorizer.transform(txts)
        return counts, df["Intenção"]
    else:
        raise ValueError("Defina o objetivo ('train' ou 'test')")

## Separando o dataset em treinamento e teste

In [414]:
# Divide em treinamento e teste, de modo estratificado, o conjunto de dados.
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_SEED)
for train_index, test_index in split.split(df, df["Intenção"]):
    strat_train_set = df.loc[train_index]
    strat_test_set = df.loc[test_index]

## Treinando o modelo

In [415]:
model = MultinomialNB()
X_train, y_train = adaptToModel(strat_train_set, objective="train")
model.fit(X_train, y_train)


MultinomialNB()

## Testando o modelo

In [416]:
X_test, y_test = adaptToModel(strat_test_set, objective="test")

In [417]:
pred = cross_val_predict(model, X_test, y_test, cv=10, n_jobs=-1)

In [418]:
output = pd.DataFrame()
output["Sentença"] = strat_test_set["Sentença"]
output["Real"] = y_test
output["Predicted"] = pred
output.head()

,Sentença,Real,Predicted
59,como vai estar o clima amanhã,Obter informações relativas ao clima,Obter informações relativas ao clima
233,apague todas as luzes vou dormir,Interagir com a luz ou o ar-condicionado,Interagir com a luz ou o ar-condicionado
112,menos luz,Interagir com a luz ou o ar-condicionado,Interagir com a luz ou o ar-condicionado
349,aumente a potência do ar-condicionado,Interagir com a luz ou o ar-condicionado,Interagir com a luz ou o ar-condicionado
294,boa noite apague a luz,Interagir com a luz ou o ar-condicionado,Interagir com a luz ou o ar-condicionado


In [419]:
score = accuracy_score(output["Real"], output["Predicted"])
print(f"Acurácia: {score * 100:.2f}%")

Acurácia: 91.11%


In [420]:
def predNewSentence(txt, predModel=model, vectorizer=v):
    counts = vectorizer.transform([txt])
    return predModel.predict(counts)[0]


In [421]:
predNewSentence("Acende a luz por favor!")

array(['Interagir com a luz ou o ar-condicionado'], dtype='<U40')

In [422]:
predNewSentence("Como tá a minha conta bancária?")

array(['Consultar saldo da conta'], dtype='<U40')

In [423]:
predNewSentence("Vai fazer sol depois de amanhã?")

array(['Obter informações relativas ao clima'], dtype='<U40')

In [424]:
predNewSentence("Marque uma consulta com o fisioterapeuta para segunda às 10 horas.")

array(['Consultar saldo da conta'], dtype='<U40')